In [1]:
import os
os.chdir ("/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA_working_11062022/")
os.getcwd()
    
!pwd

/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA_working_11062022


In [7]:
!ls -lG


total 98584
-rw-r--r--   1 neba  staff  6931030  3 Jun 03:56 Untitled1-Copy1.ipynb
-rw-r--r--   1 neba  staff    61782  3 Jun 04:46 Untitled1-Copy2.ipynb
-rw-r--r--   1 neba  staff  6923827  3 Jun 04:14 Untitled1-Copy3.ipynb
-rw-r--r--   1 neba  staff  6923831  3 Jun 03:38 Untitled1-Copy4.ipynb
-rw-r--r--   1 neba  staff  6923831  3 Jun 04:16 Untitled1-Copy5.ipynb
-rw-r--r--   1 neba  staff    62889  3 Jun 04:50 Untitled1-Copy6.ipynb
-rw-r--r--@  1 neba  staff    10831 12 Jun 20:46 Untitled1.ipynb
-rw-r--r--   1 neba  staff     4231 30 May 21:57 control_cyc1.pdb
-rw-r--r--   1 neba  staff     6118 30 May 21:58 control_cyc2.pdb
-rw-r--r--   1 neba  staff      636  2 Jun 18:01 control_fda1.pdb
-rw-r--r--   1 neba  staff     4231 30 May 21:57 cyclica1.pdb
-rw-r--r--   1 neba  staff     6118 30 May 21:58 cyclica2.pdb
-rw-r--r--   1 neba  staff     1663  2 Jun 17:56 fda901.pdb
-rw-r--r--   1 neba  staff     2058  2 Jun 17:56 fda902.pdb
-rw-r--r--   1 neba  staff     2137  2 Jun 17:56 fda903

In [8]:
#!/usr/bin/python
#
# PairWise_RMSD_smalLigs.py
 
import __main__
__main__.pymol_argv = [ 'pymol', '-qc'] # Quiet and no GUI
 
import sys, time, os, re
from pymol import cmd
import pymol
import glob
import pandas as pd

pymol.finish_launching()

directory = "/Users/neba/Desktop/CyclicaVsFDA/TEST_cyclica_FDA_working_11062022/*.pdb"
#directory2 = "/Users/neba/Desktop/CyclicaVsFDA/Results/test1/*.pdb"


# Read User Input
structurePath = glob.glob(directory)
#structurePath2 = glob.glob(directory2)

structurePath.sort()
#structurePath2.sort()
extension = re.compile( '(^.*[\/]|\.(pdb|ent|brk))' )

object_list = []
for filename in structurePath:
    object_list.append(extension.sub ('',filename))

#for filename in structurePath2:
    #object_list.append(extension.sub ('',filename))

rmsd = {}    
rmsd_list = []
for i in range (len(structurePath)):
    for j in range(1+i,len(structurePath)):
        obj_name1 = extension.sub('',structurePath[i].split("/")[-1].split(".")[0])
        cmd.load(structurePath[i],obj_name1)
        obj_name2 = extension.sub('',structurePath[j].split("/")[-1].split(".")[0])
        cmd.load(structurePath[j],obj_name2)
        
        rmsd_val = cmd.align(obj_name1, obj_name2, cycles=5, cutoff=5,
                            quiet=1, transform=0)#good ligand alignment when transform=0
        #transform=0/1: do superposition {default: 1}
                             
        
        rmsd.setdefault(obj_name1,{})[obj_name2] = rmsd_val[0],rmsd_val[1],rmsd_val[2],
        rmsd_val[3], rmsd_val[4],rmsd_val[5], rmsd_val[6]
        
        rmsd_list.append((round(rmsd_val[3],2), round(rmsd_val[0],2), round(rmsd_val[4],2),
                          round(rmsd_val[5],2), obj_name1, obj_name2)) #indexing to creat
                                                                          #columns in df
             #columns in df
        
        #Delete object and replace with new items when 
        #loop loops through directory, the delete ccommnds must
        #align with loops for good results
       
        cmd.delete(obj_name2)
    cmd.delete(obj_name1)

#Re-order table
rmsd_list.sort()

#testing
print (rmsd_list)

#Create tables with poutput pairwise rmsds
rmsd_df = pd.DataFrame(rmsd_list)

#Renaming columns to match pymol's "align" column names
rmsd_df.columns = ['RMSD_BR','RMSD_AR', '#atomsAligned',
                   'Alignment_score','Ref_Ligand', 
                   'Aligned_Ligand']

#save file to directory
rmsd_df.to_csv("rmsd900-1000_test.csv", index=False)

rmsd_df.head(50)

#The "align" tool of pymol returns a list with 7 items:
#RMSD after refinement
#Number of aligned atoms after refinement
#Number of refinement cycles
#RMSD before refinement
#Number of aligned atoms before refinement
#Raw alignment score
#Number of residues aligned
#





[(0.0, 0.0, 38, 5.0, 'control_cyc1', 'cyclica1'), (0.0, 0.0, 38, 5.0, 'control_cyc1', 'p1_79'), (0.0, 0.0, 38, 5.0, 'cyclica1', 'p1_79'), (0.0, 0.0, 57, 5.0, 'control_cyc2', 'cyclica2'), (0.0, 0.0, 57, 5.0, 'control_cyc2', 'p1_108'), (0.0, 0.0, 57, 5.0, 'cyclica2', 'p1_108'), (0.0, 0.0, 185526385, 0.0, 'p1_727', 'p1_783'), (0.11, 0.11, 3, 5.0, 'control_fda1', 'fda909'), (0.16, 0.16, 3, 5.0, 'control_fda1', 'fda901'), (0.2, 0.2, 3, 5.0, 'control_fda1', 'fda908'), (0.32, 0.32, 3, 5.0, 'control_fda1', 'fda905'), (0.49, 0.49, 3, 5.0, 'control_fda1', 'fda913'), (0.73, 0.73, 7, 5.0, 'fda913', 'fda915'), (0.82, 0.82, 6, 5.0, 'fda908', 'fda917'), (0.84, 0.84, 4, 0.0, 'control_fda1', 'p1_427'), (0.88, 0.88, 7, 5.0, 'fda913', 'fda914'), (0.95, 0.95, 6, 5.0, 'fda905', 'fda913'), (1.01, 1.01, 6, 0.0, 'fda905', 'p1_610'), (1.02, 1.02, 6, 0.0, 'fda905', 'p1_205'), (1.04, 1.04, 5, 5.0, 'fda905', 'fda906'), (1.04, 1.04, 5, 5.0, 'fda905', 'fda917'), (1.07, 1.07, 5, 5.0, 'fda905', 'fda912'), (1.07, 1.0

,RMSD_BR,RMSD_AR,#atomsAligned,Alignment_score,Ref_Ligand,Aligned_Ligand
0,0.00,0.00,38,5.0,control_cyc1,cyclica1
1,0.00,0.00,38,5.0,control_cyc1,p1_79
2,0.00,0.00,38,5.0,cyclica1,p1_79
3,0.00,0.00,57,5.0,control_cyc2,cyclica2
4,0.00,0.00,57,5.0,control_cyc2,p1_108
5,0.00,0.00,57,5.0,cyclica2,p1_108
6,0.00,0.00,185526385,0.0,p1_727,p1_783
7,0.11,0.11,3,5.0,control_fda1,fda909
8,0.16,0.16,3,5.0,control_fda1,fda901
9,0.20,0.20,3,5.0,control_fda1,fda908


In [ ]:
pd.read_csv('rmsd900-1000.csv')

In [ ]:
rmsd_df[(rmsd_df["RMSD_BR"] < 2) & (rmsd_df["Alignment_score"] > 3)
        & (rmsd_df['#atomsAligned'] > 5)].head(50)

In [ ]:
#import time for timing runs
import time
start_time = time.time()


In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
from sklearn.decomposition import TruncatedSVD
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%%time
# install kb
!pip install --quiet kb-python 

In [ ]:
%%time
# Download the data from the 10x website
!curl -O http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v3/pbmc_1k_v3_fastqs.tar

# unpack the downloaded files
!tar -xf pbmc_1k_v3_fastqs.tar

In [ ]:
!tar -czf pbmc_1k_v3_fastqs.tar pbmc_1k_v3_fastq

In [ ]:
!ls *.tar